# Explore and cluster the neighborhoods in Toronto. 

### Importing libraries.

In [1]:
import pandas as pd
import numpy as np
import requests
import io
import folium                               # map rendering library
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors

from geopy.geocoders import Nominatim       # convert an address into latitude and longitude values
from pandas.io.json import json_normalize   # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans


### Dataframe created in second part.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
        
url2 = 'http://cocl.us/Geospatial_data'
temp = requests.get(url2).content
geo_data = pd.read_csv(io.StringIO(temp.decode()))
geo_data.head()
geo_data = geo_data.rename(index=str, columns={'Postal Code':'Postcode'})
geo_data = pd.merge(df,geo_data, on='Postcode')
geo_data.head(18)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Creating map of Torronto.

### Downloading coordinates.

In [3]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Torronto are 43.653963, -79.387207.


### Drawing a map.

In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(geo_data['Latitude'], geo_data['Longitude'], geo_data['Borough'], geo_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Create a new data frame with neighborhoods in North York.

In [7]:
# @hidden_cell
CLIENT_ID =                                                                                                                                           'TACEWFTLHR54SCB2GARYSTTQIH5R13ETFZJ3LQZSAOMPO0CI' # your Foursquare ID
CLIENT_SECRET =                                                                                                                                       'W4AFT53EUXDNGDZFJD11XSULOMALB4LGYIL15BEU5TAZD0NL' # your Foursquare Secret
VERSION =                                                                                                                                             '20180605'

In [8]:
north_york_data = geo_data[geo_data['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [9]:
address = 'North York, Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude_north = location.latitude
longitude_north = location.longitude
print('The geograpical coordinate of Torronto are {}, {}.'.format(latitude_north, longitude_north))

The geograpical coordinate of Torronto are 43.7708175, -79.4132998.


### Creating Map of North York and its neighbourhood.

In [12]:
map_north_york = folium.Map(location=[latitude_north, longitude_north], zoom_start=12)

# add markers to map
for lat, lng, label in zip(north_york_data['Latitude'], north_york_data['Longitude'], north_york_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_north_york)  
    
map_north_york

### Get the top 100 venues in the neighborhood 'Hillcrest Village', from North York.

In [13]:
neighborhood_latitude = north_york_data.loc[0, 'Latitude'] # neighbourhood latitude value
neighborhood_longitude = north_york_data.loc[0, 'Longitude'] # neighbourhood longitude value

neighborhood_name = north_york_data.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Hillcrest Village" are 43.8037622, -79.3634517.


In [21]:
LIMIT = 100
radius = 500
url3 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, 
                                                                                                                           CLIENT_SECRET, 
                                                                                                                           latitude_north, 
                                                                                                                           longitude_north, 
                                                                                                                           VERSION, 
                                                                                                                           radius, 
                                                                                                                           LIMIT)

In [22]:
results = requests.get(url3).json()
results

{'meta': {'code': 200, 'requestId': '5cc59aab44f4190025444c69'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Willowdale',
  'headerFullLocation': 'Willowdale, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 52,
  'suggestedBounds': {'ne': {'lat': 43.7753175045, 'lng': -79.40707971454917},
   'sw': {'lat': 43.7663174955, 'lng': -79.41951988545084}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '563d44fccd1044ad67a744fb',
       'name': "The Captain's Boil",
       'location': {'address': '5313 Yonge St',
        'lat': 43.773255217045026,
        'lng': -79.41380541792645,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.7732552170

In [23]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(15)

,name,categories,lat,lng
0,The Captain's Boil,Seafood Restaurant,43.773255,-79.413805
1,Loblaws,Grocery Store,43.768648,-79.412597
2,Aroma Espresso Bar,Café,43.769449,-79.413081
3,Dakgogi,Korean Restaurant,43.773010,-79.413875
4,Burrito Boyz,Burrito Place,43.773054,-79.414082
5,Starbucks,Coffee Shop,43.768192,-79.413021
6,Cineplex Cinemas Empress Walk,Movie Theater,43.768625,-79.412613
7,Konjiki Ramen,Ramen Restaurant,43.766998,-79.412222
8,The Keg,Steakhouse,43.766579,-79.412131
9,Owl of Minerva,Korean Restaurant,43.773328,-79.413915


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

52 venues were returned by Foursquare.


In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues for each neighborhood in North York.

In [40]:
north_york_venues = getNearbyVenues(names=north_york_data['Neighbourhood'],
                                   latitudes=north_york_data['Latitude'],
                                   longitudes=north_york_data['Longitude'])

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [43]:
north_york_venues.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports
5,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store
6,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777548,-79.344485,Shopping Mall
7,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Hero Certified Burgers,43.777295,-79.344584,Burger Joint
8,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Apple Fairview,43.777915,-79.343561,Electronics Store
9,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Shoppers Drug Mart,43.778878,-79.344697,Pharmacy


In [44]:
north_york_venues.tail(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
235,Glencairn,43.709577,-79.445073,Hookah bar,43.707802,-79.443284,Hookah Bar
236,Glencairn,43.709577,-79.445073,Miyako sushi,43.707717,-79.447597,Asian Restaurant
237,"Downsview, North Park, Upwood Park",43.713756,-79.490074,Rustic Bakery,43.715414,-79.490300,Bakery
238,"Downsview, North Park, Upwood Park",43.713756,-79.490074,Sporty,43.716503,-79.489838,Basketball Court
239,"Downsview, North Park, Upwood Park",43.713756,-79.490074,Maple leaf park,43.716188,-79.493531,Park
240,"Downsview, North Park, Upwood Park",43.713756,-79.490074,Mika's Trim,43.714068,-79.496113,Construction & Landscaping
241,Humber Summit,43.756303,-79.565963,Pizza Monza,43.755043,-79.567195,Pizza Place
242,Humber Summit,43.756303,-79.565963,The Famous Mama Mia,43.758820,-79.570637,Empanada Restaurant
243,"Emery, Humberlea",43.724766,-79.532242,Sunset Aluminum,43.722971,-79.532220,Construction & Landscaping
244,"Emery, Humberlea",43.724766,-79.532242,Strathburn Park,43.721765,-79.532854,Baseball Field


In [45]:
north_york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
"CFB Toronto, Downsview East",4,4,4,4,4,4
Don Mills North,6,6,6,6,6,6
Downsview Central,4,4,4,4,4,4
Downsview Northwest,5,5,5,5,5,5
Downsview West,5,5,5,5,5,5
"Downsview, North Park, Upwood Park",4,4,4,4,4,4


In [46]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 109 uniques categories.


In [48]:
# one hot encoding
north_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_onehot['Neighbourhood'] = north_york_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [north_onehot.columns[-1]] + list(north_onehot.columns[:-1])
north_onehot = north_onehot[fixed_columns]

north_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
north_onehot.shape

(245, 110)

In [50]:
north_grouped = north_onehot.groupby('Neighbourhood').mean().reset_index()
north_grouped.head(10)

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.058824,0.0,...,0.058824,0.0,0.000000,0.0,0.0,0.0,0.058824,0.0,0.0,0.0
1,Bayview Village,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.250000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.00,0.038462,0.0,0.0,0.0,0.00,0.000000,0.0,...,0.038462,0.0,0.076923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CFB Toronto, Downsview East",0.0,0.25,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Don Mills North,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5,Downsview Central,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,Downsview Northwest,0.0,0.00,0.000000,0.0,0.0,0.2,0.00,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
7,Downsview West,0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.200000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
8,"Downsview, North Park, Upwood Park",0.0,0.00,0.000000,0.0,0.0,0.0,0.25,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
9,"Emery, Humberlea",0.0,0.00,0.000000,0.0,0.0,0.0,0.00,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


### Geting 10 top venues per neighborhood.

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = north_grouped['Neighbourhood']

for ind in np.arange(north_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Supermarket,Bank,Deli / Bodega,Restaurant,Diner,Sandwich Place,Shopping Mall,Pizza Place,Pharmacy
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Thai Restaurant,Pizza Place,Italian Restaurant,Sandwich Place,Comfort Food Restaurant,Café,Liquor Store,Juice Bar
3,"CFB Toronto, Downsview East",Other Repair Shop,Airport,Bus Stop,Park,Dog Run,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop
4,Don Mills North,Caribbean Restaurant,Baseball Field,Café,Basketball Court,Japanese Restaurant,Gym / Fitness Center,Furniture / Home Store,Frozen Yogurt Shop,Comfort Food Restaurant,Construction & Landscaping
5,Downsview Central,Korean Restaurant,Business Service,Food Truck,Baseball Field,Women's Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
6,Downsview Northwest,Grocery Store,Gym / Fitness Center,Athletics & Sports,Discount Store,Liquor Store,Empanada Restaurant,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
7,Downsview West,Grocery Store,Shopping Mall,Moving Target,Hotel,Bank,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
8,"Downsview, North Park, Upwood Park",Construction & Landscaping,Bakery,Basketball Court,Park,Women's Store,Electronics Store,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Deli / Bodega
9,"Emery, Humberlea",Construction & Landscaping,Baseball Field,Women's Store,Empanada Restaurant,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


### Using k-means to cluster the neighborhoods into 5 clusters.

In [69]:
north_data = north_york_data.drop(16)

kclusters = 5
north_grouped_clustering = north_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 4])

### Include kmeans.labels_ into the original North York dataframe.

In [70]:
north_merged = north_data

north_merged['Cluster Labels'] = kmeans.labels_
north_merged = north_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

north_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Dog Run,Pool,Athletics & Sports,Mediterranean Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Women's Store,Toy / Game Store,Asian Restaurant,Bus Station,Bakery,Japanese Restaurant
2,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0,Cafeteria,Women's Store,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M2N,North York,Willowdale South,43.770120,-79.408493,0,Restaurant,Coffee Shop,Ramen Restaurant,Sandwich Place,Shopping Mall,Café,Movie Theater,Middle Eastern Restaurant,Indonesian Restaurant,Japanese Restaurant
6,M2P,North York,York Mills West,43.752758,-79.400049,0,Park,Bank,Electronics Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
7,M2R,North York,Willowdale West,43.782736,-79.442259,0,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Butcher,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
8,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Fast Food Restaurant,Dog Run,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
9,M3B,North York,Don Mills North,43.745906,-79.352188,4,Caribbean Restaurant,Baseball Field,Café,Basketball Court,Japanese Restaurant,Gym / Fitness Center,Furniture / Home Store,Frozen Yogurt Shop,Comfort Food Restaurant,Construction & Landscaping


### Visualization of a cluster on the map.

In [72]:
map_clusters = folium.Map(location = [latitude_north, longitude_north], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_merged['Latitude'], north_merged['Longitude'], north_merged['Neighbourhood'], north_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine each of our 5 clusters.

In [73]:
north_merged.loc[north_merged['Cluster Labels'] == 0, north_merged.columns[[1] + list(range(2, north_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Hillcrest Village,43.803762,-79.363452,0,Golf Course,Dog Run,Pool,Athletics & Sports,Mediterranean Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
1,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Clothing Store,Fast Food Restaurant,Coffee Shop,Restaurant,Women's Store,Toy / Game Store,Asian Restaurant,Bus Station,Bakery,Japanese Restaurant
2,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Café,Bank,Japanese Restaurant,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
3,North York,"Silver Hills, York Mills",43.757490,-79.374714,0,Cafeteria,Women's Store,Electronics Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
4,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,North York,Willowdale South,43.770120,-79.408493,0,Restaurant,Coffee Shop,Ramen Restaurant,Sandwich Place,Shopping Mall,Café,Movie Theater,Middle Eastern Restaurant,Indonesian Restaurant,Japanese Restaurant
6,North York,York Mills West,43.752758,-79.400049,0,Park,Bank,Electronics Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store
7,North York,Willowdale West,43.782736,-79.442259,0,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Butcher,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
8,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Fast Food Restaurant,Dog Run,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
10,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,0,Gym,Asian Restaurant,Coffee Shop,Beer Store,Sandwich Place,Grocery Store,Fast Food Restaurant,Italian Restaurant,Japanese Restaurant,Discount Store


In [74]:
north_merged.loc[north_merged['Cluster Labels'] == 1, north_merged.columns[[1] + list(range(2, north_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Event Space,Coffee Shop,Miscellaneous Shop,Shoe Store,Boutique,Accessories Store,Vietnamese Restaurant,Gift Shop


In [75]:
north_merged.loc[north_merged['Cluster Labels'] == 2, north_merged.columns[[1] + list(range(2, north_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,Downsview West,43.739015,-79.506944,2,Grocery Store,Shopping Mall,Moving Target,Hotel,Bank,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


In [76]:
north_merged.loc[north_merged['Cluster Labels'] == 3, north_merged.columns[[1] + list(range(2, north_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,"Emery, Humberlea",43.724766,-79.532242,3,Construction & Landscaping,Baseball Field,Women's Store,Empanada Restaurant,Coffee Shop,Comfort Food Restaurant,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


In [77]:
north_merged.loc[north_merged['Cluster Labels'] == 4, north_merged.columns[[1] + list(range(2, north_merged.shape[1]))]]

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,North York,Don Mills North,43.745906,-79.352188,4,Caribbean Restaurant,Baseball Field,Café,Basketball Court,Japanese Restaurant,Gym / Fitness Center,Furniture / Home Store,Frozen Yogurt Shop,Comfort Food Restaurant,Construction & Landscaping
